# Pandas -- Series and DataFrames


Main source: https://github.com/jakevdp/PythonDataScienceHandbook/tree/master/notebooks

#### Pandas is a library for fast and efficient computation on big datasets. As in Numpy, many operations in Pandas are vectorized and thus efficient and fast.
Pandas is a newer package built on top of NumPy, and provides an efficient implementation of a DataFrame. DataFrames are essentially multidimensional arrays with attached row and column labels, and often with heterogeneous types and/or missing data. As well as offering a convenient storage interface for labeled data, Pandas implements a number of powerful data operations familiar to users of both database frameworks (-> relational algebra) and spreadsheet programs.

As we saw, NumPy's ndarray data structure provides essential features for the type of clean, well-organized data typically seen in numerical computing tasks. While it serves this purpose very well, its limitations become clear when we need more flexibility (e.g., attaching labels to data, working with missing data, etc.) and when attempting operations that do not map well to element-wise broadcasting (e.g., groupings, pivots, etc.), each of which is an important piece of analyzing the less structured data available in many forms in the world around us. Pandas, and in particular its Series and DataFrame objects, builds on the NumPy array structure and provides efficient access to these sorts of "data munging" tasks that occupy much of a data scientist's time.

In [1]:
# Just as we import numpy usually as np, we import pandas under the alias of pd. 
# We'll import numpy as well, because we'll need it often when using pandas
import numpy as np
import pandas as pd

## The Pandas Series Object
A Pandas Series is a one-dimensional array of indexed data. It can be created from a list or array as follows:

In [2]:
data = pd.Series([0.25, 5, np.NaN, 1.0])
data

0    0.25
1    5.00
2     NaN
3    1.00
dtype: float64

In [3]:
type(data)

pandas.core.series.Series

In [4]:
data.values, type(data.values)

(array([0.25, 5.  ,  nan, 1.  ]), numpy.ndarray)

In addition to the values Pandas Series have an index which is an array-like object

In [5]:
data.index, type(data.index), list(data.index)

(RangeIndex(start=0, stop=4, step=1),
 pandas.core.indexes.range.RangeIndex,
 [0, 1, 2, 3])

Like with a NumPy array, data can be accessed by the associated index via the familiar Python square-bracket notation:

In [6]:
data[1:3]

1    5.0
2    NaN
dtype: float64

In [7]:
type(data[1])

numpy.float64

### Series as generalized NumPy array

From what we've seen so far, it may look like the Series object is basically interchangeable with a one-dimensional NumPy array. The essential difference is the presence of the index: while the Numpy Array has an implicitly defined integer index used to access the values, the Pandas Series has an explicitly defined index associated with the values.

This explicit index definition gives the Series object additional capabilities. For example, the index need not be an integer, but can consist of values of any desired type. For example, if we wish, we can use strings as an index:

In [8]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'd', 'c'])
data

a    0.25
b    0.50
d    0.75
c    1.00
dtype: float64

In [9]:
data.index = list("AbCD")
data

A    0.25
b    0.50
C    0.75
D    1.00
dtype: float64

In [10]:
data["b"] == data[1] == data.values[1]

True

In [11]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=[3, 7, 2, 4])
data

3    0.25
7    0.50
2    0.75
4    1.00
dtype: float64

In [12]:
data.index

Int64Index([3, 7, 2, 4], dtype='int64')

When an explicit index is present, it is preferred! (*as long as we don't slice!*)

In [13]:
data[3]

0.25

In [14]:
data[1:3]

7    0.50
2    0.75
dtype: float64

Note that explicit indices don't need to be unique!

In [15]:
d2 = pd.concat([data, data])
d2

3    0.25
7    0.50
2    0.75
4    1.00
3    0.25
7    0.50
2    0.75
4    1.00
dtype: float64

In [16]:
d2[3]

3    0.25
3    0.25
dtype: float64

In [17]:
d2[3] = 2
d2

3    2.00
7    0.50
2    0.75
4    1.00
3    2.00
7    0.50
2    0.75
4    1.00
dtype: float64

### Series as specialized dictionary

In this way, you can think of a Pandas Series a bit like a specialization of a Python dictionary. A dictionary is a structure that maps arbitrary keys to a set of arbitrary values, and a Series is a structure which maps typed keys to a set of typed values. This typing is important: just as the type-specific compiled code behind a NumPy array makes it more efficient than a Python list for certain operations, the type information of a Pandas Series makes it much more efficient than Python dictionaries for certain operations.

The Series-as-dictionary analogy can be made even more clear by constructing a Series object directly from a Python dictionary:

In [18]:
population_dict = {'California': 38332521,
                   'Texas': 26448193,
                   'New York': 19651127,
                   'Florida': 19552860,
                   'Illinois': 12882135}
population = pd.Series(population_dict)
population

California    38332521
Texas         26448193
New York      19651127
Florida       19552860
Illinois      12882135
dtype: int64

In [19]:
population['Texas']

26448193

## The Pandas Index Object

We have seen here that both the Series and DataFrame objects contain an explicit index that lets you reference and modify data. This Index object is an interesting structure in itself, and it can be thought of as an immutable array:

In [20]:
ind = pd.Index([2, 3, 5, 7, 11])
ind

Int64Index([2, 3, 5, 7, 11], dtype='int64')

In [21]:
#ind[0] = 1

In [22]:
sr = pd.Series(0, index=ind)
sr

2     0
3     0
5     0
7     0
11    0
dtype: int64

Index objects have a name:

In [23]:
ind.names = ['indexx']
ind

Int64Index([2, 3, 5, 7, 11], dtype='int64', name='indexx')

In [24]:
sr = pd.Series(np.zeros_like(ind), index=ind)
sr

indexx
2     0
3     0
5     0
7     0
11    0
dtype: int64

In [25]:
df = pd.DataFrame(np.zeros_like(ind), index=ind, columns=['first'])
df

,first
indexx,
2,0
3,0
5,0
7,0
11,0


In [26]:
df.index.names = [None]
df

,first
2,0
3,0
5,0
7,0
11,0


Index objects also have many of the attributes familiar from NumPy arrays:

In [27]:
ind.size, ind.shape, ind.ndim, ind.dtype

(5, (5,), 1, dtype('int64'))

While viewing Indices as immutable list is natural, indices also allow for set-operations:

In [28]:
indA = pd.Index([1, 3, 5, 7, 9])
indB = pd.Index([2, 3, 5, 7, 11])

In [29]:
indA & indB

C:\Users\argha\AppData\Local\Temp/ipykernel_10504/144964795.py:1: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  indA & indB


Int64Index([3, 5, 7], dtype='int64')

In [30]:
indA | indB

C:\Users\argha\AppData\Local\Temp/ipykernel_10504/3251124211.py:1: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  indA | indB


Int64Index([1, 2, 3, 5, 7, 9, 11], dtype='int64')

In [31]:
indA ^ indB

C:\Users\argha\AppData\Local\Temp/ipykernel_10504/2832893868.py:1: FutureWarning: Index.__xor__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__xor__.  Use index.symmetric_difference(other) instead
  indA ^ indB


Int64Index([1, 2, 9, 11], dtype='int64')

<br>

# Data Indexing, Selection, and Assignment

From the numpy lecture, we already know about indexing, slicing, masking, and fancy indexing:

In [32]:
a = np.arange(16).reshape(4,4)
a

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [33]:
# Takes those values of the second and fourth column that are divisible by 3
a[:, [1, 3]][a[:, [1, 3]] % 3 == 0]

array([ 3,  9, 15])

Here we'll look at similar means of accessing and modifying values in Pandas Series and DataFrame objects. The corresponding patterns in Pandas are very similar to those of numpy, though there are a few quirks to be aware of.

We'll start with the simple case of the one-dimensional Series object, and then move on to the slightly more complicated two-dimensional DataFrame object.

## Data Selection in Series

As we saw in the previous section, a Series object acts in many ways like a one-dimensional NumPy array, and in many ways like a standard Python dictionary. If we keep these two overlapping analogies in mind, it will help us to understand the patterns of data indexing and selection in these arrays.

### Series as dictionary

Like a dictionary, the Series object provides a mapping from a collection of keys to a collection of values, which means most of the corresponding functions work just as well for them:

In [34]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

In [35]:
data.__contains__('b')

True

In [36]:
'b' in data

True

In [37]:
np.array_equal(data.keys(), data.index)

True

In [38]:
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

In [39]:
list(data.items())

[('a', 0.25), ('b', 0.5), ('c', 0.75), ('d', 1.0)]

In [40]:
data['e'] = 1.25
data

a    0.25
b    0.50
c    0.75
d    1.00
e    1.25
dtype: float64

### Series as one-dimensional array

Series builds on this dictionary-like interface and provides array-style item selection via the same basic mechanisms as NumPy arrays – that is, slices, masking, and fancy indexing. Examples of these are as follows:

In [41]:
# slicing by explicit index
data['a':'c']

a    0.25
b    0.50
c    0.75
dtype: float64

In [42]:
# slicing by implicit integer index
data[0:2] 
# Note that when slicing with an explicit index (i.e., data['a':'c']), the final index is included in the slice, 
# while when slicing with an implicit index (i.e., data[0:2]), the final index is excluded from the slice.

a    0.25
b    0.50
dtype: float64

In [43]:
(data > 0.3) & (data < 0.8)

a    False
b     True
c     True
d    False
e    False
dtype: bool

In [44]:
# masking
data[(data > 0.3) & (data < 0.8)]

b    0.50
c    0.75
dtype: float64

In [45]:
# fancy indexing
data[['a', 'e']]

a    0.25
e    1.25
dtype: float64

In [46]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=[1, 2, 3, 4])
data

1    0.25
2    0.50
3    0.75
4    1.00
dtype: float64

In [47]:
data[1:3]

2    0.50
3    0.75
dtype: float64

**If your Series has an explicit integer index, an indexing operation such as data[1] will use the explicit indices, while a slicing operation like data[1:3] will use the implicit Python-style index.**

In [48]:
data = pd.Series(['a', 'b', 'c'], index=[1, 5, 3])
data

1    a
5    b
3    c
dtype: object

In [49]:
# explicit index when indexing
data[1]

'a'

In [50]:
# implicit index when slicing
data[1:3]

5    b
3    c
dtype: object

The **loc** attribute allows indexing and slicing that *always* references the explicit index:

In [51]:
data.loc[1]

'a'

In [52]:
data.loc[1:3]

1    a
5    b
3    c
dtype: object

Note that `loc` may or may not throw Index-Errors when slicing:

In [53]:
data = pd.Series(['a', 'b', 'c'], index=[1, 5, 3])
data

1    a
5    b
3    c
dtype: object

In [54]:
try:
    
    data.loc['a':'z']
    
except KeyError:
    print(KeyError)

<class 'KeyError'>


In [55]:
try:
    
    data.loc[3:10]
    
except KeyError:
    print(KeyError)

<class 'KeyError'>


In [56]:
data = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])
data.loc[3:10]

3    b
5    c
dtype: object

In [57]:
data.loc['a':'z']

Series([], dtype: object)

The **iloc** attribute allows indexing and slicing that always references the implicit Python-style index:

In [58]:
data

1    a
3    b
5    c
dtype: object

In [59]:
data.iloc[1]

'b'

In [60]:
data.iloc[1:3]

3    b
5    c
dtype: object

Please, save yourself the pain and be always explicit about what you do -- always use ``.loc`` and ``.iloc``

In [61]:
%%bash
python -c "import this" | grep "Explicit"
#not saying that explicit indices are better than implicit ones, but that you should be explicit about what you're doing.

Couldn't find program: 'bash'


Unlike a dictionary, though, the Series also supports array-style operations such as slicing:

In [62]:
population['California':'Illinois']
# note that Illinois is included!

California    38332521
Texas         26448193
New York      19651127
Florida       19552860
Illinois      12882135
dtype: int64

# Reading Series and DataFrames

Lets have a look inside the `pokemon.csv` file and the content inside it. 
The content is quite unredable for now but this is the data we are going to use for this section of the lecture. 

In [63]:
%%bash
head Pokemon.csv

Couldn't find program: 'bash'


Pandas provides us with the function calle `read_csv` to read the csv file and take care of the data. by default it takes the first row of the csv file as column names and rest of the data belong to the columns. 

https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

In [64]:
#df = pd.read_csv("Pokemon.csv",  sep= ",", header='infer', index_col=None, skiprows=None )
df = pd.read_csv("Pokemon.csv")

Imagine someboy gave you a random dataset. You don't know any of its contents. What are the first steps you do?

with `.head()` you have take a look inside the dataset. It prints the first 5 row of the dataset by default. However, you can pass number as pameter of how many data you want to view.

In [65]:
df.head()


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [66]:
df.shape

(800, 13)

In [67]:
df.size

10400

similar thing can be done with `.tail()` fuction. the only difference is, it prints the data from the end of the dataset. 

In [68]:
df.tail()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True
799,721,Volcanion,Fire,Water,600,80,110,120,130,90,70,6,True


with `.describe()` you can print the summary of the dataset. The summary includes central tendency, 

Now what does it means? How can we interpret the results?  

as you can see in the cell avobe, the dataset has 800 entires. we can aslo verify it with `df.shape` which gives us the row and column numbers of the dataset. the row shows `count`, `mean`, `std`, `25%`, `50%`, `75%` and `max`. The coulmn has all teh columns that are possibelt to describe with in the scope of summary statistics. 

  - the **count** shows us the count of entries each column has (every column has 800 entries)
  - **mean** shows us the mean values of each column
  - **std** goes for standerddiviation. 
  - **min** values in each column
  - the **percentile** of the dataset where the data belongs to. 
  - **max** values in each column. 

In [69]:
df.describe()

,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
count,800.000000,800.00000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.00000
mean,362.813750,435.10250,69.258750,79.001250,73.842500,72.820000,71.902500,68.277500,3.32375
std,208.343798,119.96304,25.534669,32.457366,31.183501,32.722294,27.828916,29.060474,1.66129
min,1.000000,180.00000,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000,1.00000
25%,184.750000,330.00000,50.000000,55.000000,50.000000,49.750000,50.000000,45.000000,2.00000
50%,364.500000,450.00000,65.000000,75.000000,70.000000,65.000000,70.000000,65.000000,3.00000
75%,539.250000,515.00000,80.000000,100.000000,90.000000,95.000000,90.000000,90.000000,5.00000
max,721.000000,780.00000,255.000000,190.000000,230.000000,194.000000,230.000000,180.000000,6.00000


`.value_counts()` gives us the unique values in a column and the count of the values as well.

the clomun `Type 1` has only catagorical values.  so we get the output as categorical names and the count of the catagories.  

In [70]:
df["Type 1"].value_counts()

Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Electric     44
Rock         44
Dragon       32
Ground       32
Ghost        32
Dark         31
Poison       28
Steel        27
Fighting     27
Ice          24
Fairy        17
Flying        4
Name: Type 1, dtype: int64

as the colum Legendary is a boolen column, it contains only true or false values. 

In [71]:
df["Legendary"].value_counts()

False    735
True      65
Name: Legendary, dtype: int64

In [72]:
#when working with csvs you really want to make sure if you want the first column as index-column!
df = pd.read_csv("Pokemon.csv", index_col=0)
df.tail() 

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
#,,,,,,,,,,,,
719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True
721,Volcanion,Fire,Water,600,80,110,120,130,90,70,6,True


it is possible to chain multiple function calls one after another. 

In [73]:
df.reset_index().tail()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True
799,721,Volcanion,Fire,Water,600,80,110,120,130,90,70,6,True


In [74]:
df.shape

(800, 12)

In [75]:
df.reset_index().drop_duplicates(subset="#").tail()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
793,717,Yveltal,Dark,Flying,680,126,131,95,131,98,99,6,True
794,718,Zygarde50% Forme,Dragon,Ground,600,108,100,121,81,95,95,6,True
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
799,721,Volcanion,Fire,Water,600,80,110,120,130,90,70,6,True


In [76]:
df = df[df['Name'] != 'Volcanion']
df.tail()

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
#,,,,,,,,,,,,
718,Zygarde50% Forme,Dragon,Ground,600,108,100,121,81,95,95,6,True
719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


when reseting index, it creates a new index column and if there are any previous index column founds, it remains there as index column, to drop the old column, which is not in any use anymore, we can use drop function and pass the column name as parameter that we want to drop and set axis as 1 to tell pandas to drop the column. 

In [77]:
no_duplicates = df.reset_index().drop_duplicates(subset="#").reset_index().drop("index", axis=1)  
no_duplicates.tail()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
715,716,Xerneas,Fairy,NaN,680,126,131,95,131,98,99,6,True
716,717,Yveltal,Dark,Flying,680,126,131,95,131,98,99,6,True
717,718,Zygarde50% Forme,Dragon,Ground,600,108,100,121,81,95,95,6,True
718,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
719,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True


with `.to_csv()` we can save the cleaned dataframe as csv file. 


In [78]:
no_duplicates.set_index("#").to_csv('Pokemon_no_duplicates.csv')
#no_duplicates.to_excel('Pokemon_no_duplicates.xlsx', sheet_name='Sheet1')

In [79]:
%%bash
head Pokemon_no_duplicates.csv

Couldn't find program: 'bash'


with `.set_index()` we can create index with our desired column

In [80]:
gen_one = no_duplicates[no_duplicates["Generation"] == 1].set_index("#")
gen_one.tail()

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
#,,,,,,,,,,,,
147,Dratini,Dragon,NaN,300,41,64,45,50,50,50,1,False
148,Dragonair,Dragon,NaN,420,61,84,65,70,70,70,1,False
149,Dragonite,Dragon,Flying,600,91,134,95,100,100,80,1,False
150,Mewtwo,Psychic,NaN,680,106,110,90,154,90,130,1,True
151,Mew,Psychic,NaN,600,100,100,100,100,100,100,1,False


In [81]:
first_gen_dict = gen_one["Name"].to_dict()
first_gen_dict

{1: 'Bulbasaur',
 2: 'Ivysaur',
 3: 'Venusaur',
 4: 'Charmander',
 5: 'Charmeleon',
 6: 'Charizard',
 7: 'Squirtle',
 8: 'Wartortle',
 9: 'Blastoise',
 10: 'Caterpie',
 11: 'Metapod',
 12: 'Butterfree',
 13: 'Weedle',
 14: 'Kakuna',
 15: 'Beedrill',
 16: 'Pidgey',
 17: 'Pidgeotto',
 18: 'Pidgeot',
 19: 'Rattata',
 20: 'Raticate',
 21: 'Spearow',
 22: 'Fearow',
 23: 'Ekans',
 24: 'Arbok',
 25: 'Pikachu',
 26: 'Raichu',
 27: 'Sandshrew',
 28: 'Sandslash',
 29: 'Nidoran♀',
 30: 'Nidorina',
 31: 'Nidoqueen',
 32: 'Nidoran♂',
 33: 'Nidorino',
 34: 'Nidoking',
 35: 'Clefairy',
 36: 'Clefable',
 37: 'Vulpix',
 38: 'Ninetales',
 39: 'Jigglypuff',
 40: 'Wigglytuff',
 41: 'Zubat',
 42: 'Golbat',
 43: 'Oddish',
 44: 'Gloom',
 45: 'Vileplume',
 46: 'Paras',
 47: 'Parasect',
 48: 'Venonat',
 49: 'Venomoth',
 50: 'Diglett',
 51: 'Dugtrio',
 52: 'Meowth',
 53: 'Persian',
 54: 'Psyduck',
 55: 'Golduck',
 56: 'Mankey',
 57: 'Primeape',
 58: 'Growlithe',
 59: 'Arcanine',
 60: 'Poliwag',
 61: 'Poliwhirl'

In [82]:
[str(key)+" : "+str(val) for index, (key, val) in enumerate(first_gen_dict.items()) if index < 9]

['1 : Bulbasaur',
 '2 : Ivysaur',
 '3 : Venusaur',
 '4 : Charmander',
 '5 : Charmeleon',
 '6 : Charizard',
 '7 : Squirtle',
 '8 : Wartortle',
 '9 : Blastoise']

**Documentation!**

There are really really many arguments for this function, suiting all of your needs!

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

# Ufuncs and Aggregation

## Aggregation in Pandas

Aggregations are functions, where one or more dimensions of data are collapsed onto a single value, like the `max`, `sum` or `mean`- functions.

Stat-operations generally *exclude* missing data.

### For Series

In [83]:
a = np.arange(7)
ser = pd.Series(a**2, index=a)
ser

0     0
1     1
2     4
3     9
4    16
5    25
6    36
dtype: int32

In [84]:
ser.sum()
#mean(), median(), min(), max(), ...

91

### For DataFrames

In [85]:
df = pd.DataFrame({'A': a**2,
                   'B': a**3})
df

,A,B
0,0,0
1,1,1
2,4,8
3,9,27
4,16,64
5,25,125
6,36,216


In [86]:
df.mean()

A    13.0
B    63.0
dtype: float64

In [87]:
df.mean(axis=0)

A    13.0
B    63.0
dtype: float64

In [88]:
df.mean(axis='rows')

A    13.0
B    63.0
dtype: float64

In [89]:
df.mean(axis='columns')

0      0.0
1      1.0
2      6.0
3     18.0
4     40.0
5     75.0
6    126.0
dtype: float64

The following table summarizes some other built-in Pandas aggregations:

| Aggregation              | Description                     |
|--------------------------|---------------------------------|
| ``count()``              | Total number of items           |
| ``first()``, ``last()``  | First and last item             |
| ``mean()``, ``median()`` | Mean and median                 |
| ``min()``, ``max()``     | Minimum and maximum             |
| ``std()``, ``var()``     | Standard deviation and variance |
| ``mad()``                | Mean absolute deviation         |
| ``prod()``               | Product of all items            |
| ``sum()``                | Sum of all items                |

These are all methods of ``DataFrame`` and ``Series`` objects.

## Ufuncs


We know Ufuncs already from Numpy: It are vectorized functions that change all values of an array simultaneously. 

Pandas does the same, with a nice twist: for unary operations like negation and trigonometric functions, these ufuncs will *preserve index and column labels* in the output, and for binary operations such as addition and multiplication, Pandas will automatically *align indices* when passing the objects to the ufunc.


This means that keeping the context of data and combining data from different sources –both potentially error-prone tasks with raw NumPy arrays– become essentially foolproof ones with Pandas.

In [90]:
rng = np.random.RandomState(0)
df = pd.DataFrame(rng.randint(0, 10, (3, 4)),
                  columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,5,0,3,3
1,7,9,3,5
2,2,4,7,6


In [91]:
np.exp(df)

,A,B,C,D
0,148.413159,1.000000,20.085537,20.085537
1,1096.633158,8103.083928,20.085537,148.413159
2,7.389056,54.598150,1096.633158,403.428793


### UFuncs: Index Alignment

For binary operations on two ``Series`` or ``DataFrame`` objects, Pandas will align indices in the process of performing the operation.
This is very convenient when working with incomplete data.

In [92]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662,
                  'California': 423967}, name='area')
population = pd.Series({'California': 38332521, 'Texas': 26448193,
                        'New York': 19651127}, name='population')
area

Alaska        1723337
Texas          695662
California     423967
Name: area, dtype: int64

In [93]:
population

California    38332521
Texas         26448193
New York      19651127
Name: population, dtype: int64

In [94]:
# area & population
area.index.intersection(population.index)

Index(['Texas', 'California'], dtype='object')

In [95]:
"divide" in dir(pd.DataFrame)

True

In [96]:
population/area

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

In [97]:
popdens = population.divide(area, fill_value=0)
popdens

Alaska         0.000000
California    90.413926
New York            inf
Texas         38.018740
dtype: float64

In [98]:
popdens = popdens.replace([np.inf, -np.inf], np.nan)
popdens.dropna()

Alaska         0.000000
California    90.413926
Texas         38.018740
dtype: float64

In [99]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)),
                 columns=list('AB'))
A

,A,B
0,12,1
1,6,7


In [100]:
B = pd.DataFrame(rng.randint(0, 20, (3, 3)),
                 columns=list('ABC'))
B

,A,B,C
0,14,17,5
1,13,8,9
2,19,16,19


In [101]:
A+B

,A,B,C
0,26.0,18.0,NaN
1,19.0,15.0,NaN
2,NaN,NaN,NaN


In [102]:
A.add(B, fill_value=0)

,A,B,C
0,26.0,18.0,5.0
1,19.0,15.0,9.0
2,19.0,16.0,19.0


### More Index-Alignment

In [103]:
df = pd.DataFrame({'a': np.random.randint(3, size=10)}, index=np.arange(1, 20, 2))
df

,a
1,0
3,1
5,0
7,0
9,1
11,2
13,2
15,0
17,0
19,0


Let's add a new column to this DataFrame!

In [104]:
tmp = pd.Series([1]*len(df.index))
tmp

0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
dtype: int64

In [105]:
# df.align will unify the indices
df.index.union(tmp.index)

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 13, 15, 17, 19], dtype='int64')

In [106]:
left_aligned, right_aligned = df.align(tmp, axis=0)
left_aligned

,a
0,NaN
1,0.0
2,NaN
3,1.0
4,NaN
5,0.0
6,NaN
7,0.0
8,NaN
9,1.0


In [107]:
right_aligned

0     1.0
1     1.0
2     1.0
3     1.0
4     1.0
5     1.0
6     1.0
7     1.0
8     1.0
9     1.0
11    NaN
13    NaN
15    NaN
17    NaN
19    NaN
dtype: float64

In [108]:
test_left_a, test_right_a = df.align(tmp, axis=0)
test_left_b, test_right_b = tmp.align(df, axis=0)
print(test_left_a.equals(test_right_b))
print(test_right_a.equals(test_left_b))

True
True


In [109]:
tmp = pd.Series([0]*len(df.index), index=df.index)
tmp

1     0
3     0
5     0
7     0
9     0
11    0
13    0
15    0
17    0
19    0
dtype: int64

In [110]:
df['new'] = tmp
df

,a,new
1,0,0
3,1,0
5,0,0
7,0,0
9,1,0
11,2,0
13,2,0
15,0,0
17,0,0
19,0,0


## .agg()

If you want to apply more than one operation (ufunc/aggregation), use `.agg()`:

In [111]:
df = pd.DataFrame([[1, 2, 3],
                   [4, 5, 6],
                   [7, 8, 9],
                   [np.nan, np.nan, np.nan]],
                  columns=['A', 'B', 'C'])
df

,A,B,C
0,1.0,2.0,3.0
1,4.0,5.0,6.0
2,7.0,8.0,9.0
3,NaN,NaN,NaN


In [112]:
df.agg(['sum', np.min])

,A,B,C
sum,12.0,15.0,18.0
amin,1.0,2.0,3.0


In [113]:
#you can also use different aggregations for different columns: 
df.agg({'A' : ['sum', 'min'], 'B' : ['min', 'max']})

,A,B
sum,12.0,NaN
min,1.0,2.0
max,NaN,8.0


In [114]:
#also works for ufuncs:
df.agg({'A' : 'exp', 'B' : [np.exp, 'sqrt']})

A            B          
           exp          exp      sqrt
0     2.718282     7.389056  1.414214
1    54.598150   148.413159  2.236068
2  1096.633158  2980.957987  2.828427
3          NaN          NaN       NaN

## apply()

While some ufuncs (like cumsum or exp) are pre-defined by pandas, the method `apply` can be used to run an arbitrary function on all elements of a Series or DataFrame.

In [115]:
a = np.arange(7)
df = pd.DataFrame({'A': a**2,
                   'B': a**3})
df

,A,B
0,0,0
1,1,1
2,4,8
3,9,27
4,16,64
5,25,125
6,36,216


In [116]:
df.cumsum()

,A,B
0,0,0
1,1,1
2,5,9
3,14,36
4,30,100
5,55,225
6,91,441


In [117]:
df.apply(np.cumsum)

,A,B
0,0,0
1,1,1
2,5,9
3,14,36
4,30,100
5,55,225
6,91,441


Using Lambda-functions, we can combine `apply` with arbitrary functions. Note that the argument of the function is always an entire column of the dataset.

In [118]:
df.apply(lambda x: print(x, end='\n\n'))

0     0
1     1
2     4
3     9
4    16
5    25
6    36
Name: A, dtype: int32

0      0
1      1
2      8
3     27
4     64
5    125
6    216
Name: B, dtype: int32



A    None
B    None
dtype: object

In [119]:
df['A'] + 1

0     1
1     2
2     5
3    10
4    17
5    26
6    37
Name: A, dtype: int32

In [120]:
df.apply(lambda x: x+1)

,A,B
0,1,1
1,2,2
2,5,9
3,10,28
4,17,65
5,26,126
6,37,217


In [121]:
def my_more_complex_func(ser):
    res = [9]
    for elem in ser:
        print(elem if elem > 16 else -elem)
        res.append(elem if elem > 16 else -elem)
    return res

In [122]:
df.apply(my_more_complex_func)

0
-1
-4
-9
-16
25
36
0
-1
-8
27
64
125
216


,A,B
0,9,9
1,0,0
2,-1,-1
3,-4,-8
4,-9,27
5,-16,64
6,25,125
7,36,216


In [123]:
df

,A,B
0,0,0
1,1,1
2,4,8
3,9,27
4,16,64
5,25,125
6,36,216


In [124]:
df.apply(lambda x: x.max() - x.min())

A     36
B    216
dtype: int32

Note that `apply` works for both DataFrames and Series!

In [125]:
df["A"].apply(lambda x: print(x,"\n"))

0 

1 

4 

9 

16 

25 

36 



0    None
1    None
2    None
3    None
4    None
5    None
6    None
Name: A, dtype: object

In [126]:
df["A_normed"] = df["A"].apply(lambda x: x/df["A"].max())
df

,A,B,A_normed
0,0,0,0.000000
1,1,1,0.027778
2,4,8,0.111111
3,9,27,0.250000
4,16,64,0.444444
5,25,125,0.694444
6,36,216,1.000000


It can be very useful to use dictionaries in combination with the apply-function!

In [127]:
z_moves = {"Normal": "Breakneck Blitz", "Fighting": "All-Out Pummeling", "Flying": "Supersonic Skystrike", "Poison": "Acid Downpour", "Ground": "Tectonic Rage", "Rock": "Continental Crush", "Bug": "Savage Spin-Out", "Ghost": "Never-Ending Nightmare",
"Steel": "Corkscrew Crash", "Fire": "Inferno Overdrive", "Water": "Hydro Vortex", "Grass": "Bloom Doom", "Electric": "Gigavolt Havoc", "Psychic": "Shattered Psyche", "Ice": "Subzero Slammer", "Dragon": "Devastating Drake", "Dark": "Black Hole Eclipse", "Fairy": "Twinkle Tackle"}
df = pd.read_csv("Pokemon.csv")
df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [128]:
df["Z-Move"] = df["Type 1"].apply(lambda x:z_moves[x])
df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Z-Move
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,Bloom Doom
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,Bloom Doom
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,Bloom Doom
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False,Bloom Doom
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False,Inferno Overdrive


Using `apply`, we can also convert a list of Series into a DataFrame, by making the individual columns to Series:

In [129]:
s = pd.Series([ ['Red', 'Green', 'White'], ['Red', 'Black'], ['Yellow']]) 
print(type(s))
s

<class 'pandas.core.series.Series'>


0    [Red, Green, White]
1           [Red, Black]
2               [Yellow]
dtype: object

In [130]:
pd.Series([1, 2, 3])

0    1
1    2
2    3
dtype: int64

In [131]:
df = s.apply(pd.Series)
print(type(df))
df

<class 'pandas.core.frame.DataFrame'>


,0,1,2
0,Red,Green,White
1,Red,Black,NaN
2,Yellow,NaN,NaN


**Note on speed:**

According to ([1]), `apply()` is twice as fast as looping through a df's `iterrows()`, and 8 times as fast as looping over python lists.

Note however, that while `apply()` is much faster at looping over the rows of your DataFrame/Series (by taking advantage of a number of internal optimizations, such as using iterators in Cython), it still inherently loops through rows. Whatever you're applying, you're still executing it once for every row. So, wherever you can make use of vectorized Ufuncs, do so - that is far more optimized and parallelized - for ([1]) exchanging the haversine distance formula with it's vectorized counterpart led to a 50-fold-decrease in time!

\(1): https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6


[1]: https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6 

### Exercise

Write a Pandas program to convert a Series of lists to one Series ("flatten" the Series).

In [132]:
s = pd.Series([['Red', 'Green', 'White'], ['Red', 'Black'], ['Yellow']])
s

0    [Red, Green, White]
1           [Red, Black]
2               [Yellow]
dtype: object

In [133]:
# Sample Output: 
print(pd.Series(['Red', 'Green', 'White', 'Red', 'Black', 'Yellow']))
# Your code here


0       Red
1     Green
2     White
3       Red
4     Black
5    Yellow
dtype: object


### Solution

In [134]:
s.apply(pd.Series).stack().reset_index(drop=True)

0       Red
1     Green
2     White
3       Red
4     Black
5    Yellow
dtype: object

# Group-By

## Split-Apply-Combine

While simple operations are already pre-defined by pandas, custom aggregations and operations can be performed via **group-by**. The group-by operation can be described as having the following steps:

* **Splitting** the data into groups based on some criteria (breaking up and grouping depending on the value of a key)
* **Applying** a function to each group independently (aggregation, transformation, filtering, ...)
* **Combining** the results into a data structure

A typical example, for where the *apply* is a summerization aggregation, is illustrated here:

![](split-apply-combine.png)

In [135]:
tmp = np.array([list("ABCABC"), np.arange(1,7)]).T
tmp

array([['A', '1'],
       ['B', '2'],
       ['C', '3'],
       ['A', '4'],
       ['B', '5'],
       ['C', '6']], dtype='<U11')

In [136]:
df = pd.DataFrame(tmp, columns=["key", "data"])
df["data"] = pd.to_numeric(df["data"])
df

,key,data
0,A,1
1,B,2
2,C,3
3,A,4
4,B,5
5,C,6


In [137]:
df.groupby("key")

Note that what is returned is not a set of `DataFrames`, but a `DataFrameGroupBy` object. This object is where the magic is: you can think of it as a special view of the `DataFrames`, which is poised to dig into the groups but does no actual computation until the aggregation is applied. This "lazy evaluation" approach means that common aggregates can be implemented very efficiently in a way that is almost transparent to the user.

To produce a result, we can apply an aggregate to this `DataFrameGroupBy` object, which will perform the appropriate apply/combine steps to produce the desired result:

In [138]:
df.groupby("key").sum().reset_index()

,key,data
0,A,5
1,B,7
2,C,9


In [139]:
df.groupby("key")["data"].sum()
# we can do column indexing just like on a normal DataFrame

key
A    5
B    7
C    9
Name: data, dtype: int64

Sometimes making a proper DataFrame out of the result requires some index name adjustments. There are usually multiple ways and it is necessary to put some thought into what you want to end up with in each situation.

In [140]:
# Sometimes making a proper DataFrame out of the result requires some index name adjustments
print(pd.DataFrame(df.groupby("key")["data"].sum().rename_axis(None)))
print(df.groupby("key")["data"].sum().reset_index())

   data
A     5
B     7
C     9
  key  data
0   A     5
1   B     7
2   C     9


### Iteration over groups

The ``GroupBy`` object supports direct iteration over the groups, returning each group as a ``Series`` or ``DataFrame``:

In [141]:
df

,key,data
0,A,1
1,B,2
2,C,3
3,A,4
4,B,5
5,C,6


In [142]:
for (key, _) in df.groupby("key"):
    print(key)
    
print()
for (_, group) in df.groupby("key"):
    print(group, "\n")

A
B
C

  key  data
0   A     1
3   A     4 

  key  data
1   B     2
4   B     5 

  key  data
2   C     3
5   C     6 



### Dispatch methods

Any method not explicitly implemented by the ``GroupBy`` object will be passed through and called on the groups, whether they are ``DataFrame`` or ``Series`` objects.
For example, you can use the ``describe()`` method of ``DataFrame``s to perform a set of aggregations that describe each group in the data:

In [143]:
df.describe()

,data
count,6.000000
mean,3.500000
std,1.870829
min,1.000000
25%,2.250000
50%,3.500000
75%,4.750000
max,6.000000


In [144]:
df.groupby("key").describe()

data                                         
    count mean      std  min   25%  50%   75%  max
key                                               
A     2.0  2.5  2.12132  1.0  1.75  2.5  3.25  4.0
B     2.0  3.5  2.12132  2.0  2.75  3.5  4.25  5.0
C     2.0  4.5  2.12132  3.0  3.75  4.5  5.25  6.0

### Exercise

The given dataset contains a column `Region` as well as a column `Pop. Density`. Write a snippet that takes as argument the dataframe containing all the countries, and returns a `Series` mapping regions to the average Population density of its countries.

In [145]:
countries = pd.read_csv('countries.csv', index_col=0)
countries.head()

,Country,Subcontinent,Region,In EU,Population,Area,Pop. Density,Coastline,Net migration,Infant mortality,...,Phones,Arable,Crops,Other,Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Afghanistan,West & Central Asia,ASIA (EX. NEAR EAST),False,31056997,1.677019e+06,47.96,0.00,23.06,163.07,...,3.22,12.13,0.22,87.65,1.0,46.60,20.34,0.380,0.240,0.380
1,Albania,Europe,EASTERN EUROPE,False,3581655,7.445703e+04,124.59,1.26,-4.93,21.52,...,71.20,21.09,4.42,74.49,3.0,15.11,5.22,0.232,0.188,0.579
2,Algeria,North Africa,NORTHERN AFRICA,False,32930091,6.168683e+06,13.83,0.04,-0.39,31.00,...,78.10,3.22,0.25,96.53,1.0,17.14,4.61,0.101,0.600,0.298
3,American Samoa,Oceania,OCEANIA,False,57794,5.154080e+02,290.42,58.29,-20.71,9.27,...,259.54,10.00,15.00,75.00,2.0,22.46,3.27,NaN,NaN,NaN
4,Andorra,Europe,WESTERN EUROPE,False,71201,1.212115e+03,152.14,0.00,6.60,4.05,...,497.18,2.22,0.00,97.78,3.0,8.71,6.25,NaN,NaN,NaN


In [146]:
countries.groupby("Region")["Pop. Density"].mean().rename_axis(None).sort_values(ascending=False)

ASIA (EX. NEAR EAST)    1264.819286
WESTERN EUROPE           952.042857
NEAR EAST                427.078750
NORTHERN AMERICA         260.872000
LATIN AMER. & CARIB      136.191778
OCEANIA                  131.182857
EASTERN EUROPE           100.890833
SUB-SAHARAN AFRICA        92.259020
C.W. OF IND. STATES       56.700833
BALTICS                   39.833333
NORTHERN AFRICA           38.935000
Name: Pop. Density, dtype: float64

## Aggregate, filter, transform, apply

So far, we focused on aggregation for the apply operation, but there are more options available.
In particular, ``GroupBy`` objects have ``aggregate()``, ``filter()``, ``transform()``, and ``apply()`` methods that efficiently implement a variety of useful operations before combining the grouped data.

For the purpose of the following subsections, we'll use this ``DataFrame``:

In [147]:
def create_df():
    rng = np.random.RandomState(0)
    df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                       'data1': range(6),
                       'data2': rng.randint(0, 10, 6)},
                       columns = ['key', 'data1', 'data2'])
    return df
    
df = create_df()
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


### Aggregation

While we used some *aggregate-functions* already, the function `aggregate` (or ``agg``) is the explicit version thereof.  
It can take a string, a function, or a list thereof, and compute all the aggregates at once.

In [148]:
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [149]:
df.groupby('key').aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

Another useful pattern is to pass a dictionary mapping (old) column names to operations to be applied on that column:

In [150]:
df.groupby('key').agg({'data1': 'min',
                       'data2': 'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


In [151]:
df.groupby('key').aggregate({'data1': np.sum,
                             'data2': lambda x: np.std(x, ddof=1)})

,data1,data2
key,,
A,3,1.414214
B,5,4.949747
C,7,4.242641


### Named aggregation

To support column-specific aggregation with control over the *output* column names, pandas accepts so-called named aggregation in `GroupBy.agg()`, where:

* The keywords are the output column names
* The values are tuples whose first element is the column to select and the second element is the aggregation to apply to that column. 
    * Alternatively, you can use the pandas.NamedAgg namedtuple with the fields ['column', 'aggfunc'] to make it clearer what the arguments are. 

In [152]:
animals = pd.DataFrame({'kind': ['cat', 'dog', 'cat', 'dog'],
                        'height': [9.1, 6.0, 9.5, 34.0],
                        'weight': [7.9, 7.5, 9.9, 198.0]})
animals

,kind,height,weight
0,cat,9.1,7.9
1,dog,6.0,7.5
2,cat,9.5,9.9
3,dog,34.0,198.0


In [153]:
assert int(pd.__version__[0]) >= 1, 'Your version of pandas is too old for this!'

In [154]:
animals.groupby("kind").agg(
       min_height = pd.NamedAgg(column='height', aggfunc='min'), # using NamedAgg
       max_height = ('height', 'max'), # less explicit alternative
       average_weight = pd.NamedAgg(column='weight', aggfunc=np.mean),
    )

,min_height,max_height,average_weight
kind,,,
cat,9.1,9.5,8.90
dog,6.0,34.0,102.75


### Filtering

A filtering operation allows you to drop data based on the group properties.
For example, we might want to keep all groups in which the standard deviation is larger than some critical value:

In [155]:
def filter_func(x):
    print(x,"\n")
    return x['data2'].std() > 4

df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [156]:
df.groupby('key').std()

,data1,data2
key,,
A,2.12132,1.414214
B,2.12132,4.949747
C,2.12132,4.242641


In [157]:
df.groupby('key').filter(filter_func)
#note that this is not an aggregate - the result has the shape of the original DataFrame, just with certain lines left out!

  key  data1  data2
0   A      0      5
3   A      3      3 

  key  data1  data2
1   B      1      0
4   B      4      7 

  key  data1  data2
2   C      2      3
5   C      5      9 



,key,data1,data2
1,B,1,0
2,C,2,3
4,B,4,7
5,C,5,9


### The apply() method

The ``apply()`` method lets you apply an arbitrary function to the group results.
The function should take a ``DataFrame``, and return either a Pandas object (e.g., ``DataFrame``, ``Series``) or a scalar; the combine operation will be tailored to the type of output returned.

First, remember our ``apply`` from before:

In [158]:
create_df()

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [159]:
df = create_df()
df["data1"] = df["data1"].apply(lambda x: x/df["data1"].max())
df

,key,data1,data2
0,A,0.0,5
1,B,0.2,0
2,C,0.4,3
3,A,0.6,3
4,B,0.8,7
5,C,1.0,9


Keep in mind that ``groupby`` only returns a *view of the original DataFrame*.  
Here is an ``apply()`` that normalizes the (grouped) first column with respect to each groups maximum:

In [160]:
import warnings; warnings.filterwarnings('ignore')
try:
    del newdf
except:
    pass

In [161]:
df = create_df()
maxvals = df.groupby('key')["data1"].max()
print(maxvals, '\n\n\n')
for key, group in df.groupby('key'):
    group["data1"] /= maxvals[key]
    try:
        newdf = newdf.append(group) #appending to dataframes is bad style!
    except:
        newdf = group.copy()
    print(newdf, '\n')

newdf

key
A    3
B    4
C    5
Name: data1, dtype: int64 



  key  data1  data2
0   A    0.0      5
3   A    1.0      3 

  key  data1  data2
0   A   0.00      5
3   A   1.00      3
1   B   0.25      0
4   B   1.00      7 

  key  data1  data2
0   A   0.00      5
3   A   1.00      3
1   B   0.25      0
4   B   1.00      7
2   C   0.40      3
5   C   1.00      9 



,key,data1,data2
0,A,0.00,5
3,A,1.00,3
1,B,0.25,0
4,B,1.00,7
2,C,0.40,3
5,C,1.00,9


In [162]:
def norm_by_data1(x):
    # x is a DataFrame of group values
    x['data1'] /= x['data1'].max()
    return x

df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [163]:
df.groupby('key').apply(norm_by_data1)

,key,data1,data2
0,A,0.00,5
1,B,0.25,0
2,C,0.40,3
3,A,1.00,3
4,B,1.00,7
5,C,1.00,9
